In [1]:
from langchain.document_loaders import PyPDFLoader
import nest_asyncio
import re
from sentence_transformers import SentenceTransformer
import faiss

c:\Users\Dell\miniconda3\envs\langchain\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Function to split text into chunks of specified size with overlap
def split_text_with_overlap(text, chunk_size=450, overlap=50):
    words = text.split()
    chunks = []


    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))
    return chunks

In [6]:
nest_asyncio.apply()

# Load your PDF file
pdf_path = "../data/GOT_synopsis.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
full_text = " ".join([doc.page_content for doc in docs])
doc_texts = split_text_with_overlap(full_text, chunk_size=90, overlap=20)

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(doc_texts)

c:\Users\Dell\miniconda3\envs\langchain\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
import faiss
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)  # Add vectors to the index

In [9]:

faiss.write_index(index, 'faiss_index_file.index')


In [10]:
#to import the saved index
#index = faiss.read_index('faiss_index_file.index')